In [23]:
import pandas as pd
import numpy as np
import re
import time
from urllib import parse
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import warnings
warnings.filterwarnings(action='ignore')

import os
import sys
import urllib.request
import json

In [24]:
### [1] URL 생성관련 함수
def file_to_df():
    df = pd.read_excel('apple_iphone_url.xlsx')
    df['inch'] = df['inch'].astype(str)
    df['germany_inch'] = df['inch'].apply(lambda x: re.sub("\.",",",x))
    df['russia_rom'] = df['rom'].apply(lambda x: re.sub("gb","",x))
    
    return df
    
def url_maker(df):
    df['us_url'] = "https://www.apple.com/shop/buy-iphone/"+df['family_model']+"/"+df['inch']+"-inch-display-"+df['rom']+"-"+df['color']+"-unlocked"
    df['uk_url'] = "https://www.apple.com/uk/shop/buy-iphone/"+df['family_model']+"/"+df['inch']+"-inch-display-"+df['rom']+"-"+df['color']
    df['china_url'] = "https://www.apple.com.cn/shop/buy-iphone/"+df['family_model']+"/"+df['china_code']+"/A"
    
    df['japan_url'] = 0
    df['germany_url'] = 0
    df['russia_url'] = 0

    for i in range(len(df)):
        df['japan_url'][i] = "https://www.apple.com/jp/shop/buy-iphone/"+df['family_model'][i]+"/"+df['inch'][i]+\
                    parse.quote(df['japan_code_inch'][i])+"-"+df['rom'][i]+"-"+parse.quote(df['japan_code_color'][i])
        df['germany_url'][i] = "https://www.apple.com/de/shop/buy-iphone/"+df['family_model'][i]+"/"+df['germany_inch'][i]+"%22-display-"+\
                               df['rom'][i]+"-"+parse.quote(df['germany_code'][i]) 

        if df['family_model'][i] == "iphone-se":
            df['russia_url'][i] = "https://www.apple.com/ru/shop/buy-iphone/"+df['family_model'][i]+"/"+df['germany_inch'][i]+"-"+\
                                    "%D0%B4%D1%8E%D0%B9%D0%BC%D0%BE%D0%B2%D1%8B%D0%B9-%D0%B4%D0%B8%D1%81%D0%BF%D0%BB%D0%B5%D0%B9-"+\
                                   df['russia_rom'][i]+"%D0%B3%D0%B1-"+parse.quote(df['russia_code'][i])
        else: 
             df['russia_url'][i] = "https://www.apple.com/ru/shop/buy-iphone/"+df['family_model'][i]+"/"+\
                                    "%D0%B4%D0%B8%D1%81%D0%BF%D0%BB%D0%B5%D0%B9-"+\
                                    df['germany_inch'][i]+"-"+\
                                    "%D0%B4%D1%8E%D0%B9%D0%BC%D0%B0-"+\
                                   df['russia_rom'][i]+"%D0%B3%D0%B1-"+parse.quote(df['russia_code'][i])       

    df['india_url'] = "https://www.indiaistore.com/"+df['model']+"/"
    
    return df

### [2]크롤링 관련 코드
def select_no(browser):
    try:
        no_answer = browser.find_element_by_class_name('as-inlinetradeup-selector')
        no_answer.click()
    except: pass
    #print("click No")
          
def get_delivery_date(browser,post_code):
    #1. No 선택했는지 확인    
    browser.implicitly_wait(10)
    time.sleep(2)
    delivery_date_list = browser.find_elements_by_class_name("as-purchaseinfo-dudeinfo-deliverymsg")
    delivery_date_li=[]

    for i in range(len(delivery_date_list)):
        delivery_date_li.append(delivery_date_list[i].text)
        #print(delivery_date_list[i])
        #print(delivery_date_li)

    delivery_date = " / ".join(delivery_date_li)
    print(delivery_date)
    return delivery_date

def get_price_delivery(browser, url, country,i,post_code):
    browser.get(url)
    time.sleep(1)
    
    select_no(browser)
    delivery_date = get_delivery_date(browser,post_code)
    price = browser.find_element_by_class_name('localnav-price-content').text
    price_delivery = price+"|"+delivery_date
    return price_delivery

#네이버파파고API
def tranlator_papago(text, src_lang, dest_lang):
    
    
    client_id = "Wf_OvBKP9uonTFxvBe83" 
    client_secret = "kWuIR7oLiZ" 

    data = "source=" + src_lang + "&target=" + dest_lang + "&text=" + text
    url = "https://openapi.naver.com/v1/papago/n2mt"
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request, data=data.encode("utf-8"))
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        # print(response_body.decode('utf-8'))

        #json 형 변환
        res = json.loads(response_body.decode('utf-8'))
        
    else:
        print("Error Code:" + rescode)
    return res['message']['result']['translatedText']

In [25]:
# <1> 기본 DataFrame을 세팅한다.
df = file_to_df()
df = url_maker(df)
#df.isnull().sum()
browser = webdriver.Chrome()

In [40]:
 url_list =[('usa','us_url','10001'),('uk','uk_url','sw1a2aa'), ('china', 'china_url','100000'), 
           ('japan', 'japan_url',''), ('germany','germany_url','10115'), 
           ('russia','russia_url','101000'), ('india', 'india_url')]

# <2> 나라를 선택하고 진행 - 0번째 url 받아서 우펀코드 설정
con_num = 5    #<-----------------[입력] usa:0, uk:1, china:2, japan:3, germany:4(화면켜기), russia:5, india:6

country = url_list[con_num][0]
res_col = url_list[con_num][0]+"_res"
col_nm = url_list[con_num][1]
post_code = url_list[con_num][2]
url = df[col_nm][0] #---> us_url, uk_url, china_url, japan_url, germany_url, russia_url, india_url 

#브라우저 열기  --> 우편코드 입력해주어야함  <<<---- 
browser.get(url)
df[res_col] = 0
print(country)

russia


In [41]:
# <3> 우편코드를 설정했다면, 이제 크롤링 GoGo!!!

#for i in range(14, 63):
for i in range(len(df['model'])):
    print(i, end = " ")
    url = df[col_nm][i]
    price_delivery_list = get_price_delivery(browser, url,country,i,post_code)
    df[res_col][i] = price_delivery_list
    
display(df.head(2))

0 Пн 11 Янв – Бесплатно
1 Пн 11 Янв – Бесплатно
2 Пн 11 Янв – Бесплатно
3 Пн 11 Янв – Бесплатно
4 Пн 11 Янв – Бесплатно
5 Пн 11 Янв – Бесплатно
6 Пн 11 Янв – Бесплатно
7 Пн 11 Янв – Бесплатно
8 Пн 11 Янв – Бесплатно
9 Пн 11 Янв – Бесплатно
10 Пн 11 Янв – Бесплатно
11 Пн 11 Янв – Бесплатно
12 Пн 11 Янв – Бесплатно
13 Пн 11 Янв – Бесплатно
14 Пн 11 Янв – Бесплатно
15 Пн 11 Янв – Бесплатно
16 Пн 11 Янв – Бесплатно
17 Пн 11 Янв – Бесплатно
18 Пн 11 Янв – Бесплатно
19 Пн 11 Янв – Бесплатно
20 Пн 11 Янв – Бесплатно
21 Пн 11 Янв – Бесплатно
22 Пн 11 Янв – Бесплатно
23 Пн 11 Янв – Бесплатно
24 Пн 11 Янв – Бесплатно
25 Пн 11 Янв – Бесплатно
26 Пн 11 Янв – Бесплатно
27 Пн 11 Янв – Бесплатно
28 Пн 11 Янв – Бесплатно
29 Пн 11 Янв – Бесплатно
30 27 Янв - 3 Фев – Бесплатно
31 27 Янв - 3 Фев – Бесплатно
32 27 Янв - 3 Фев – Бесплатно
33 27 Янв - 3 Фев – Бесплатно
34 27 Янв - 3 Фев – Бесплатно
35 27 Янв - 3 Фев – Бесплатно
36 27 Янв - 3 Фев – Бесплатно
37 27 Янв - 3 Фев – Бесплатно
38 27 Янв - 3 Фев – 

,No,model,family_model,inch,rom,color,china_code,japan_code_inch,japan_code_color,germany_code,...,japan_url,germany_url,russia_url,india_url,russia_res,usa_res,uk_res,china_res,japan_res,germany_res
0,1,iphone-12,iphone-12,6.1,64gb,white,MGGN3CH,インチディスプレイ,ホワイト,weiß,...,https://www.apple.com/jp/shop/buy-iphone/iphon...,https://www.apple.com/de/shop/buy-iphone/iphon...,https://www.apple.com/ru/shop/buy-iphone/iphon...,https://www.indiaistore.com/iphone-12/,79 990.00 pyб.|Пн 11 Янв – Бесплатно,$34.54/mo. or $829.00|Today 10:30am - 12:30pm ...,£33.29/mo. or £799.00**|Tue 5 Jan – Free,RMB 262/月或 RMB 6299|星期四 2020/12/31 – 免费,"85,800円（税別）|木 2020/12/31 – 送料無料","36,51 € /Monat oder 876,30 €**|Di 5 Jan – Kost..."
1,2,iphone-12,iphone-12,6.1,64gb,black,MGGM3CH,インチディスプレイ,ブラック,schwarz,...,https://www.apple.com/jp/shop/buy-iphone/iphon...,https://www.apple.com/de/shop/buy-iphone/iphon...,https://www.apple.com/ru/shop/buy-iphone/iphon...,https://www.indiaistore.com/iphone-12/,79 990.00 pyб.|Пн 11 Янв – Бесплатно,$34.54/mo. or $829.00|Dec 31 - Jan 4 – Free,£33.29/mo. or £799.00**|Tue 5 Jan – Free,RMB 262/月或 RMB 6299|星期四 2020/12/31 – 免费,"85,800円（税別）|木 2020/12/31 – 送料無料","36,51 € /Monat oder 876,30 €**|Di 5 Jan – Kost..."


In [42]:
today_date = "12.29" #오늘일자 입력
df.to_excel('iphone_New_before_trans_'+today_date+'.xlsx')
df = pd.read_excel('iphone_New_before_trans_'+today_date+'.xlsx')

In [43]:
# <1> result 결과만 가져와서 새로운 DataFrame 생성
df_b = df[['usa_res', 'uk_res','china_res','japan_res','germany_res','russia_res']]
df_b['china_res'].value_counts()[:2]

RMB 283/月或 RMB 6799|星期四 2020/12/31 – 免费    10
RMB 250/月或 RMB 5999|星期四 2020/12/31 – 免费     5
Name: china_res, dtype: int64

In [44]:
# <2> "|"를 기준으로 가격과 Delivery Date 구분
df_b['us_price'] = df_b['usa_res'].apply(lambda x : x.split(" or ")[1].split("|")[0])
df_b['us_delivery'] = df_b['usa_res'].apply(lambda x : x.split(" or ")[1].split("|")[1])

df_b['uk_price'] = df_b['uk_res'].apply(lambda x : x.split(" or ")[1].replace("**","").split("|")[0])
df_b['uk_delivery'] = df_b['uk_res'].apply(lambda x : x.split(" or ")[1].replace("**","").split("|")[1])

#df_b['ch_price'] = df_b['china_res'].apply(lambda x : x.split("或\n")[1].split("|")[0])
#df_b['ch_delivery'] = df_b['china_res'].apply(lambda x : x.split("或\n")[1].split("|")[1])
df_b['ch_price'] = df_b['china_res'].apply(lambda x : x.split("月或 ")[1].split("|")[0])
df_b['ch_delivery'] = df_b['china_res'].apply(lambda x : x.split("月或 ")[1].split("|")[1])

df_b['ja_price'] = df_b['japan_res'].apply(lambda x : x.replace("（税別）","").split("|")[0])
df_b['ja_delivery'] = df_b['japan_res'].apply(lambda x : x.replace("（税別）","").split("|")[1])

df_b['de_price'] = df_b['germany_res'].apply(lambda x : x.split(" oder ")[1].replace("**","").split("|")[0])
df_b['de_delivery'] = df_b['germany_res'].apply(lambda x : x.split(" oder ")[1].replace("**","").split("|")[1])

df_b['ru_price'] = df_b['russia_res'].apply(lambda x : x.split("|")[0])
df_b['ru_delivery'] = df_b['russia_res'].apply(lambda x : x.split("|")[1])

df_b[['ch_del_tr', 'ja_del_tr' , 'de_del_tr', 'ru_del_tr']]=0

for i in range(len(df_b)):
    print(i, end="\r")
    chinese_word = df_b['ch_delivery'][i]
    if chinese_word=="": pass
    else: chinese_translated = tranlator_papago(chinese_word, 'zh-CN', 'en')
    df_b['ch_del_tr'][i] = chinese_translated
    
    japanese_word = df_b['ja_delivery'][i]
    if japanese_word=="": pass
    else: japanese_translated = tranlator_papago(japanese_word, 'ja', 'en')
    df_b['ja_del_tr'][i] = japanese_translated 
    
    try:
        germany_word = df_b['de_delivery'][i]
        if germany_word=="": pass
        else: germany_translated =  tranlator_papago(germany_word, 'de', 'ko')
        df_b['de_del_tr'][i] = germany_translated
    except: df_b['de_del_tr'][i]=0
    #print(chinese_translated,'\n', japanese_translated,'\n', germany_translated,'\n', russia_translated)

In [48]:
df_b.to_excel("translated2_"+today_date+".xlsx")
df_b[['ch_del_tr','ja_del_tr','de_del_tr','ru_del_tr']][:10]

,ch_del_tr,ja_del_tr,de_del_tr,ru_del_tr
0,Thursday 20/12/31 – Free,"Thursday, December 31, 2020 – Free shipping","1월 5일, 무료",0
1,Thursday 20/12/31 – Free,"Thursday, December 31, 2020 – Free shipping","1월 5일, 무료",0
2,Thursday 20/12/31 – Free,"Thursday, December 31, 2020 – Free shipping","1월 5일, 무료",0
3,Thursday 20/12/31 – Free,"Thursday, December 31, 2020 – Free shipping","1월 5일, 무료",0
4,Thursday 20/12/31 – Free,"Thursday, December 31, 2020 – Free shipping","1월 5일, 무료",0
5,Thursday 20/12/31 – Free,"Thursday, December 31, 2020 – Free shipping","1월 5일, 무료",0
6,Thursday 20/12/31 – Free,"Thursday, December 31, 2020 – Free shipping","1월 5일, 무료",0
7,Thursday 20/12/31 – Free,"Thursday, December 31, 2020 – Free shipping","1월 5일, 무료",0
8,Thursday 20/12/31 – Free,"Thursday, December 31, 2020 – Free shipping","1월 5일, 무료",0
9,Thursday 20/12/31 – Free,"Thursday, December 31, 2020 – Free shipping","1월 5일, 무료",0
